In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'geum_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

#ddday = 31
#mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    #df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
#                               colum_idx[idx], interpolation=interpolation_option[idx],
#                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2013.xlsx', '현도_2014.xlsx', '현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2013.xlsx', '대청호_2014.xlsx', '대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2013.xlsx', '장계_2014.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2013.xlsx
data/geum/자동/현도_2014.xlsx
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2013-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/대청호_2013.xlsx
data/geum/자동/대청호_2014.xlsx
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_201

1/1 [==============================] - 0s 20ms/step - gen_loss: 11.0004 - disc_loss: 0.3696 - rmse: 0.4220 - val_loss: 0.2899
Epoch 49/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8647 - disc_loss: 0.3412 - rmse: 0.3185 - val_loss: 0.3691
Epoch 50/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.1717 - disc_loss: 0.3661 - rmse: 0.3468 - val_loss: 0.5924
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1517 - disc_loss: 0.3350 - rmse: 0.2925 - val_loss: 0.3599
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7271 - disc_loss: 0.3427 - rmse: 0.3385 - val_loss: 0.5221
Epoch 53/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.5866 - disc_loss: 0.3679 - rmse: 0.3129 - val_loss: 0.4785
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3701 - disc_loss: 0.3449 - rmse: 0.3034 - val_loss: 0.3448
Epoch 55/2000
1/1 [=====================

Epoch 107/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 28.5799 - disc_loss: 0.3115 - rmse: 0.5071 - val_loss: 0.2312
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0873 - disc_loss: 0.3055 - rmse: 0.3070 - val_loss: 0.2688
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 25.8504 - disc_loss: 0.2971 - rmse: 0.4763 - val_loss: 0.2507
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1212 - disc_loss: 0.3018 - rmse: 0.3394 - val_loss: 0.2914
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2023 - disc_loss: 0.2901 - rmse: 0.3073 - val_loss: 0.3160
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2133 - disc_loss: 0.2672 - rmse: 0.3280 - val_loss: 0.2412
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8065 - disc_loss: 0.2813 - rmse: 0.2847 - val_loss: 0.2688
Epoch 114/2000
1/1

1/1 [==============================] - 0s 20ms/step - gen_loss: 17.8197 - disc_loss: 0.2823 - rmse: 0.3026 - val_loss: 0.2334
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5206 - disc_loss: 0.2695 - rmse: 0.2520 - val_loss: 0.3255
Epoch 167/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 28.4274 - disc_loss: 0.2696 - rmse: 0.5693 - val_loss: 0.2542
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0872 - disc_loss: 0.2633 - rmse: 0.2680 - val_loss: 0.2975
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.4907 - disc_loss: 0.2786 - rmse: 0.4222 - val_loss: 0.2186
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 17.2206 - disc_loss: 0.2460 - rmse: 0.2838 - val_loss: 0.2413
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3773 - disc_loss: 0.2582 - rmse: 0.4081 - val_loss: 0.2769
Epoch 172/2000
1/1 [===========

1/1 [==============================] - 0s 20ms/step - gen_loss: 16.3933 - disc_loss: 0.2336 - rmse: 0.3782 - val_loss: 0.3024
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7083 - disc_loss: 0.2308 - rmse: 0.2387 - val_loss: 0.2578
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1506 - disc_loss: 0.2353 - rmse: 0.2514 - val_loss: 0.2364
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0615 - disc_loss: 0.2499 - rmse: 0.2407 - val_loss: 0.2385
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6716 - disc_loss: 0.2480 - rmse: 0.3276 - val_loss: 0.2319
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9688 - disc_loss: 0.2559 - rmse: 0.2369 - val_loss: 0.2619
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6861 - disc_loss: 0.2530 - rmse: 0.2605 - val_loss: 0.2240
Epoch 230/2000
1/1 [==============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3604 - disc_loss: 0.1573 - rmse: 0.2106 - val_loss: 0.2047
Epoch 1558/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.1190 - disc_loss: 0.1490 - rmse: 0.2328 - val_loss: 0.1726
Epoch 1559/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5004 - disc_loss: 0.1664 - rmse: 0.1802 - val_loss: 0.2015
Epoch 1560/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0247 - disc_loss: 0.1555 - rmse: 0.2223 - val_loss: 0.1913
Epoch 1561/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2885 - disc_loss: 0.1520 - rmse: 0.2140 - val_loss: 0.1973
Epoch 1562/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6509 - disc_loss: 0.1538 - rmse: 0.1952 - val_loss: 0.2278
Epoch 1563/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8655 - disc_loss: 0.1536 - rmse: 0.2235 - val_loss: 0.5155
Epoch 1564/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9209 - disc_loss: 0.1576 - rmse: 0.2265 - val_loss: 0.1775
Epoch 1616/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.5387 - disc_loss: 0.1596 - rmse: 0.5272 - val_loss: 0.2031
Epoch 1617/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3907 - disc_loss: 0.1481 - rmse: 0.1894 - val_loss: 0.2276
Epoch 1618/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6072 - disc_loss: 0.1596 - rmse: 0.2142 - val_loss: 0.1810
Epoch 1619/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6334 - disc_loss: 0.1635 - rmse: 0.2112 - val_loss: 0.2525
Epoch 1620/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6988 - disc_loss: 0.1577 - rmse: 0.1948 - val_loss: 0.2047
Epoch 1621/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1831 - disc_loss: 0.1461 - rmse: 0.1748 - val_loss: 0.3613
Epoch 1622/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.7861 - disc_loss: 0.1643 - rmse: 0.2287 - val_loss: 0.2078
Epoch 1674/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0797 - disc_loss: 0.1639 - rmse: 0.2097 - val_loss: 0.2758
Epoch 1675/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9243 - disc_loss: 0.1684 - rmse: 0.2175 - val_loss: 0.2297
Epoch 1676/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3882 - disc_loss: 0.1467 - rmse: 0.2001 - val_loss: 0.2060
Epoch 1677/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9461 - disc_loss: 0.1502 - rmse: 0.1965 - val_loss: 0.4519
Epoch 1678/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5844 - disc_loss: 0.1638 - rmse: 0.1902 - val_loss: 0.1902
Epoch 1679/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5529 - disc_loss: 0.1550 - rmse: 0.1756 - val_loss: 0.2026
Epoch 1680/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.6664 - disc_loss: 0.1599 - rmse: 0.1989 - val_loss: 0.3947
Epoch 1732/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9017 - disc_loss: 0.1574 - rmse: 0.2859 - val_loss: 0.1691
Epoch 1733/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6657 - disc_loss: 0.1576 - rmse: 0.1898 - val_loss: 0.2030
Epoch 1734/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9700 - disc_loss: 0.1545 - rmse: 0.2195 - val_loss: 0.2737
Epoch 1735/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.9115 - disc_loss: 0.1638 - rmse: 0.2119 - val_loss: 0.1956
Epoch 1736/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1807 - disc_loss: 0.1530 - rmse: 0.1821 - val_loss: 0.1787
Epoch 1737/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.0229 - disc_loss: 0.1662 - rmse: 0.5051 - val_loss: 0.1802
Epoch 1738/2000
1/1 [=======

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.7660 - disc_loss: 0.1526 - rmse: 0.1956 - val_loss: 0.2127
Epoch 1790/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8771 - disc_loss: 0.1568 - rmse: 0.2002 - val_loss: 0.1707
Epoch 1791/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3371 - disc_loss: 0.1574 - rmse: 0.1647 - val_loss: 0.2047
Epoch 1792/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.4140 - disc_loss: 0.1698 - rmse: 0.4673 - val_loss: 0.2899
Epoch 1793/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6069 - disc_loss: 0.1511 - rmse: 0.1694 - val_loss: 0.2016
Epoch 1794/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0039 - disc_loss: 0.1542 - rmse: 0.2237 - val_loss: 0.2087
Epoch 1795/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7828 - disc_loss: 0.1624 - rmse: 0.2193 - val_loss: 0.2310
Epoch 1796/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [71]:
print(real_df_all.shape)
rnn_target_column = 'do'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(70128, 36)
do


In [72]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (49089, 36) val_df.shape :  (14026, 36) test_df.shape: (7013, 36)


In [73]:
#fake_df.shape[0]/8760

In [74]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [77]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  2
out_num_features :  1


In [78]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [79]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/geum_auto_13/do/


In [80]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [81]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.9019 - mean_absolute_error: 0.7204 - nse: 0.2738
Epoch 00001: val_nse improved from -inf to 0.03169, saving model to save/best_model.h5
10/10 [==============================] - 10s 1s/step - loss: 0.9019 - mean_absolute_error: 0.7204 - nse: 0.2738 - val_loss: 0.5822 - val_mean_absolute_error: 0.6637 - val_nse: 0.0317
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.2169 - mean_absolute_error: 0.3610 - nse: 0.8256
Epoch 00002: val_nse improved from 0.03169 to 0.58281, saving model to save/best_model.h5
10/10 [==============================] - 10s 980ms/step - loss: 0.2169 - mean_absolute_error: 0.3610 - nse: 0.8256 - val_loss: 0.2504 - val_mean_absolute_error: 0.4188 - val_nse: 0.5828
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.1241 - mean_absolute_error: 0.2635 - nse: 0.8963
Epoch 00003: val_nse improved from 0.58281 to 0.64364, saving model to save/best_model.h5
10/10 [

Epoch 22/50
10/10 [==============================] - ETA: 0s - loss: 0.0351 - mean_absolute_error: 0.1411 - nse: 0.9704
Epoch 00022: val_nse did not improve from 0.84559
10/10 [==============================] - 10s 963ms/step - loss: 0.0351 - mean_absolute_error: 0.1411 - nse: 0.9704 - val_loss: 0.1194 - val_mean_absolute_error: 0.2474 - val_nse: 0.8022
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.0357 - mean_absolute_error: 0.1436 - nse: 0.9710
Epoch 00023: val_nse did not improve from 0.84559
10/10 [==============================] - 10s 989ms/step - loss: 0.0357 - mean_absolute_error: 0.1436 - nse: 0.9710 - val_loss: 0.1436 - val_mean_absolute_error: 0.2542 - val_nse: 0.7607
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.0324 - mean_absolute_error: 0.1371 - nse: 0.9744
Epoch 00024: val_nse did not improve from 0.84559
10/10 [==============================] - 10s 972ms/step - loss: 0.0324 - mean_absolute_error: 0.1371 - nse: 0.9744 - v

10/10 [==============================] - ETA: 0s - loss: 0.0281 - mean_absolute_error: 0.1286 - nse: 0.9779

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
